# Running the ZPVC code for an NMR isotropic value from a Gaussian calculation

Requirements:

 - Need the gradient and property (NMR) calculations of all the diaplaced structures.
 
The output files must all be in some readable directory so we can use the `vibrav.util.open_files.get_data()` function. In this example the gradients and properties are calculated separately. It may be the case that for the user the gradients and properties can be calculated in the same calculation simplifying things slightly. One would still need the gradient and property values.

In [8]:
from vibrav.util.open_files import get_all_data
from exatomic import gaussian
from vibrav.zpvc import ZPVC
from vibrav.base import resource
import tarfile
import os
import shutil
import numpy as np

Extract the data files from the resource data files.

In [2]:
with tarfile.open(resource('g16-nitromalonamide-zpvc-data.tar.xz'), 'r:xz') as tar:
    tar.extractall()
parent = os.getcwd()
os.chdir('g16-nitromalonamide-zpvc-data')

## Get the gradient and property data frames.

We get only the isotropic values of the NMR shielding tensor since we are interested in the observable quantity. In addition, we must specify which atom the property is to be calculated for.

In [3]:
grad = get_all_data(cls=gaussian.Output, path='output', property='gradient',
                    f_start='nitromal-grad', f_end='.out')
prop = get_all_data(cls=gaussian.Output, path='output', property='nmr_shielding',
                    f_start='nitromal-prop', f_end='.out')
prop = prop.groupby('atom').get_group(0)[['isotropic', 'file']]

Check if any of the gradient or property calculations did not finish correctly. This is also done in the ZPVC code, however, this can give a better idea of which are actually missing.

In [4]:
print("Missing property calculations:")
print(np.setdiff1d(range(79), prop.file.values))
print("Missing gradient calculations:")
print(np.setdiff1d(range(79), grad.file.values))

Missing property calculations:
[]
Missing gradient calculations:
[]


## Run the ZPVC code for temperatures `[0, 100, 200, 300, 400, 600]`.

In [5]:
zpvc = ZPVC(config_file='va.conf')

In [7]:
zpvc.zpvc(gradient=grad, property=prop, temperature=[0, 100, 200, 300, 400, 600])

Print out the main results

In [9]:
zpvc.zpvc_results

,property,zpvc,zpva,tot_anharm,tot_curva,temp
0,13.9329,-2.837661,11.095239,-3.690552,0.852892,0
1,13.9329,-2.755005,11.177895,-3.620932,0.865927,100
2,13.9329,-2.596616,11.336284,-3.510535,0.913919,200
3,13.9329,-2.427606,11.505294,-3.410414,0.982808,300
4,13.9329,-2.250905,11.681995,-3.315094,1.064189,400
5,13.9329,-1.902325,12.030575,-3.155685,1.253360,600


Clean up the resuorce data files.

In [10]:
os.chdir(parent)
shutil.rmtree('g16-nitromalonamide-zpvc-data')